# DeepSeek Debugging Notebook
This notebook isolates and debugs all DeepSeek-specific logic, including device and padding issues.

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import os
model_name = 'DeepSeek-R1-Distill-Qwen-1.5B'  # Change as needed
model_dir = '../../models'  # Adjust if needed
device = 'cuda:1' if torch.cuda.device_count() > 1 else 'cuda:0'
print(f'Using device: {device}')

Using device: cuda:0


In [2]:
# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    os.path.join(model_dir, model_name),
    torch_dtype=torch.float32,
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(os.path.join(model_dir, model_name), trust_remote_code=True)
model.to(device)
print('Model loaded and moved to', next(model.parameters()).device)

HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '../../models/DeepSeek-R1-Distill-Qwen-1.5B'. Use `repo_type` argument if needed.

In [ ]:
# DeepSeek/Qwen-specific padding handling
if ('deepseek' in model_name.lower()) or ('deepseek' in getattr(model.config, 'model_type', '').lower()) or ('qwen' in model_name.lower()) or ('qwen' in getattr(model.config, 'model_type', '').lower()):
    if tokenizer.pad_token is None or tokenizer.pad_token_id is None or tokenizer.pad_token_id < 0:
        tokenizer.pad_token = tokenizer.eos_token
        tokenizer.pad_token_id = tokenizer.eos_token_id
    tokenizer.padding_side = 'left'
print('pad_token:', tokenizer.pad_token)
print('pad_token_id:', tokenizer.pad_token_id)
print('padding_side:', tokenizer.padding_side)

In [ ]:
# Prepare a minimal batch for testing
questions = ["What is the capital of France?", "What is 2+2?"]
inputs = tokenizer(questions, return_tensors='pt', padding='longest', return_token_type_ids=False)
inputs['input_ids'] = inputs['input_ids'].to(device)
if 'attention_mask' in inputs:
    inputs['attention_mask'] = inputs['attention_mask'].to(device)
print('input_ids device:', inputs['input_ids'].device)
if 'attention_mask' in inputs:
    print('attention_mask device:', inputs['attention_mask'].device)

In [ ]:
# Try a minimal generation with verbose error catching
try:
    print('Model device before generation:', next(model.parameters()).device)
    gen_tokens = model.generate(**inputs, max_new_tokens=10, do_sample=False)
    print('Generation successful!')
    print('Generated tokens:', gen_tokens)
    print('Decoded:', tokenizer.batch_decode(gen_tokens, skip_special_tokens=True))
except Exception as e:
    import traceback
    print('Generation failed!')
    traceback.print_exc()
    print('Model device:', next(model.parameters()).device)
    print('input_ids device:', inputs['input_ids'].device)
    if 'attention_mask' in inputs:
        print('attention_mask device:', inputs['attention_mask'].device)
